## 유튜브 음성 데이터 전처리 및 metadata.txt 파일 만들기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/speech_dt/')

In [ ]:
import numpy as np
import librosa
import soundfile as sf

In [ ]:
!pip install pydub

In [ ]:
# 오디오 파일들 초 세기
from pydub import AudioSegment

secs = []

for audio_file in os.listdir('./wavs/'):
  audio = AudioSegment.from_file(f'./wavs/{audio_file}') # 오디오 파일 로드

  # 총 재생 시간 계산
  total_duration = len(audio) / 1000  # 밀리초 단위를 초 단위로 변환
  secs.append(total_duration)

  # 결과 출력
  print(f"{audio_file} 파일의 총 재생 시간: {total_duration}초")

#### 1. 유튜브 음성 파일 6초 단위로 자르기

In [ ]:
def split_wav(data, sample_rate, start, end):
  start *= sample_rate
  end *= sample_rate

  return data[round(start):round(end)]

In [ ]:
for sec, audio_file in zip(secs, os.listdir('./wavs/')):
  if sec % 6 != 0:
    iter = sec // 6 + 1
  else:
    iter = sec // 6

  audio_start = 0
  audio_end = 0
  audio_name = audio_file.split(".")[0]
  audio_file = f'./wavs/{audio_file}'

  for i in range(1, int(iter)):
    if sec - audio_end > 6:
      audio_end = 6 * i
    else:
      audio_end = sec
    save_file = f'./edited_wavs/{audio_name}_{i}.wav'
    y, sr = librosa.load(audio_file, sr=22050)
    edited_data = split_wav(y, sr, audio_start, audio_end)
    sf.write(save_file, edited_data, sr, subtype='PCM_16')
    audio_start = audio_end
  print(f'{audio_file} editing complete.')

#### 2. 유튜브 음성 전사

In [ ]:
!pip install SpeechRecognition
import os

In [ ]:
import speech_recognition as sr

def wav_to_text(wav_file):

    recognizer = sr.Recognizer()

    with sr.AudioFile(wav_file) as source:
        audio_data = recognizer.record(source)

    try:
        text = recognizer.recognize_google(audio_data, language='ko-KR')
        return text
    except sr.UnknownValueError:
        return 0
    except sr.RequestError as e:
        print(f"[오류 발생: {e}]")
        return 1

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/speech_dt/')

In [ ]:
if not os.path.exists('./STT'):
  os.mkdir('./STT')

In [ ]:
# STT

wav_dir = './edited_wavs/'
with open('STT/speaker_dt_text.txt', 'w', encoding='utf-8') as file_out:
    for wav_file in os.listdir(wav_dir):
        text = wav_to_text(wav_dir+wav_file)
        if text == 0:
          print("[음성 불명확] "+ wav_file)
        elif text == 1:
          print("[오류 발생] " + wav_file)
        else:
          file_out.write(f'{wav_file}: {text}\n')
          print(wav_file+" 변환 완료")

print("모든 파일 변환 완료")

[음성 불명확] sp_1_6.wav
sp_1_5.wav 변환 완료
sp_1_2.wav 변환 완료
sp_1_3.wav 변환 완료
sp_1_4.wav 변환 완료
sp_1_1.wav 변환 완료
sp_1_18.wav 변환 완료
sp_1_17.wav 변환 완료
sp_1_15.wav 변환 완료
sp_1_16.wav 변환 완료
sp_1_14.wav 변환 완료
sp_1_13.wav 변환 완료
sp_1_12.wav 변환 완료
sp_1_11.wav 변환 완료
sp_1_9.wav 변환 완료
sp_1_10.wav 변환 완료
sp_1_8.wav 변환 완료
sp_1_7.wav 변환 완료
sp_1_27.wav 변환 완료
sp_1_28.wav 변환 완료
sp_1_29.wav 변환 완료
sp_1_26.wav 변환 완료
[음성 불명확] sp_1_25.wav
sp_1_24.wav 변환 완료
sp_1_23.wav 변환 완료
[음성 불명확] sp_1_22.wav
sp_1_21.wav 변환 완료
sp_1_20.wav 변환 완료
sp_1_19.wav 변환 완료
sp_1_37.wav 변환 완료
sp_1_36.wav 변환 완료
[음성 불명확] sp_1_35.wav
sp_1_34.wav 변환 완료
sp_1_33.wav 변환 완료
sp_1_32.wav 변환 완료
sp_1_31.wav 변환 완료
sp_1_30.wav 변환 완료
sp_1_49.wav 변환 완료
sp_1_48.wav 변환 완료
sp_1_47.wav 변환 완료
sp_1_46.wav 변환 완료
sp_1_45.wav 변환 완료
sp_1_44.wav 변환 완료
[음성 불명확] sp_1_43.wav
sp_1_42.wav 변환 완료
sp_1_41.wav 변환 완료
sp_1_40.wav 변환 완료
sp_1_39.wav 변환 완료
sp_1_38.wav 변환 완료
sp_1_74.wav 변환 완료
sp_1_73.wav 변환 완료
sp_1_72.wav 변환 완료
sp_1_71.wav 변환 완료
sp_1_70.wav 변환 완료
sp_1_69.wav 변환 완료
sp_1

#### 3. 음성 파일 전사 제대로 안된 파일 걸러내기

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/speech_dt/')

In [ ]:
# 음성 불명확한 데이터 삭제
# wav파일 wavs_dir로 이동
import shutil
# make metadata.txt
#filelists/wavs/wav파일명.wav|전사된 텍스트

TargetText = '[음성 불명확]'
wavs_dir = "wavs/"
rawtxtfile = '/content/drive/MyDrive/Colab Notebooks/speech_dt/STT/speaker_dt_text.txt'
fromMovefiledir = '/content/drive/MyDrive/Colab Notebooks/speech_dt/edited_wavs/'
toMovedir = '/content/drive/MyDrive/Colab Notebooks/speech_dt/' + wavs_dir
# wav 파일 잘못옮겼을때 되돌려놓기
# for file in os.listdir(toMovedir):
#   shutil.move(f'{toMovedir}{file}', fromMovefiledir) # 데이터로 쓸 wav 파일만 이동


with open(rawtxtfile, 'r', encoding='utf8') as metadata:
  lines = metadata.readlines()
  for line in lines:
    String = line.split('\n')
    String = String[0].split(': ')

    # 음성 불명확 아닌 wav 파일만 wavs_dir로 이동시키고, metadata.txt에 기록
    if String[1] != TargetText:
      shutil.move(f'{fromMovefiledir}{String[0]}', toMovedir) # 데이터로 쓸 wav 파일만 이동
      with open("metadata.txt", 'a', encoding='utf8') as f2:
        f2.write(f'{wavs_dir}{String[0]}|{String[1]}\n')

file_list = os.listdir(toMovedir)
print("총 wav 파일 수: ",len(file_list))

총 wav 파일 수:  776
